In this notebook Longitude and latitude columns are added in order for further visualisations and processing. This proccess take a lot of time, therefore it is done in separate notebook and columns added before all further manipulation with the dataset.

In [1]:
from geopy.geocoders import Nominatim
import pandas as pd
import time

In [2]:
df = pd.read_parquet("../data/raw/pollution_dataset.parquet", engine="pyarrow")
df.head()

,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.2,21,NaN
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.2,23,25.0
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.2,21,NaN
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.2,23,25.0
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.6,23,NaN


In [ ]:


# 1 — Extract unique addresses
unique_addresses = pd.DataFrame(df["Address"].unique(), columns=["Address"])
print("Unique addresses:", len(unique_addresses))

# 2 — Prepare columns
unique_addresses["Latitude"] = None
unique_addresses["Longitude"] = None

# 3 — Initialize geocoder
geolocator = Nominatim(user_agent="my_geocoder_app")

def geocode_address(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        return None, None
    except:
        return None, None

# 4 — Loop with delay
coords = []
for addr in unique_addresses["Address"]:
    lat, lon = geocode_address(addr)
    coords.append((lat, lon))
    time.sleep(1)   # Respect Nominatim rate limits

unique_addresses["Latitude"], unique_addresses["Longitude"] = zip(*coords)

# 5 — Save to parquet
unique_addresses.to_parquet("../data/raw/coordinates.parquet", index=False)
print("Saved geocoded addresses to address_coordinates.parquet")

Unique addresses: 204
Saved geocoded addresses to address_coordinates.parquet


In [ ]:
unique_addresses.head()
print(unique_addresses.isna().sum())
print(unique_addresses.shape)

,Address,Latitude,Longitude
0,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,NaN,NaN
1,2857 N MILLER RD-S SCOTTSDALE STN,NaN,NaN
2,"1237 S. BEVERLY , TUCSON",32.206263,-110.879259
3,2956-A TREAT BOULEVARD,37.936691,-122.028736
4,5551 BETHEL ISLAND RD,38.003272,-121.641333


In [ ]:
"""
df = pd.merge(df, unique_addresses, on="Address", how="left")
df.to_parquet("../data/preprocessed/pollution_dataset_enriched.parquet", index=False)
"""